## Step 1:Install dependencies and configure environment

In [0]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  python-snappy
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 10.8 kB of archives.
After this operation, 39.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-snappy amd64 0.5-1.1build2 [10.8 kB]
Fetched 10.8 kB in 1s (14.5 kB/s)
Selecting previously unselected package python-snappy.
(Reading database ... 130912 files and directories currently installed.)
Preparing to unpack .../python-snappy_0.5-1.1build2_amd64.deb ...
Unpacking python-snappy (0.5-1.1build2) ...
Setting up python-snappy (0.5-1.1build2) ...


In [0]:
!pip install -q tensorflow_data_validation

     |████████████████████████████████| 2.7MB 2.8MB/s 
     |████████████████████████████████| 2.8MB 33.6MB/s 
     |████████████████████████████████| 174kB 41.8MB/s 
     |████████████████████████████████| 51kB 16.8MB/s 
     |████████████████████████████████| 51kB 16.9MB/s 
     |████████████████████████████████| 1.0MB 37.8MB/s 
     |████████████████████████████████| 153kB 44.7MB/s 
     |████████████████████████████████| 81kB 21.3MB/s 
     |████████████████████████████████| 143kB 39.0MB/s 
     |████████████████████████████████| 92kB 23.8MB/s 
     |████████████████████████████████| 102kB 24.8MB/s 
     |████████████████████████████████| 163kB 34.5MB/s 
     |████████████████████████████████| 92kB 22.8MB/s 
ERROR: google-cloud-storage 1.16.1 has requirement google-cloud-core<2.0dev,>=1.0.0, but you'll have google-cloud-core 0.29.1 which is incompatible.
ERROR: multiprocess 0.70.8 has requirement dill>=0.3.0, but you'll have dill 0.2.9 which is incompatible.
ERROR: tensorflow-trans

## Step 2: Import libraries

In [0]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

from __future__ import print_function

## Step 3: Descriptive Analisys on Dataset


In [0]:
dataset = pd.read_csv("pollution-small.csv")

In [0]:
dataset.shape

(2188, 5)

In [0]:
training_data = dataset[:1600]

In [0]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [0]:
test_set = dataset[1600:]

In [0]:
test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Step 3b: Descriptive Analisys and validation with TFDV

### Generate the descriptive analisys on Dataset


In [0]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=training_data)

W0805 15:32:15.928714 140011396999040 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensorflow_transform/analyzers.py:948: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0805 15:32:17.189505 140011396999040 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensorflow_transform/analyzers.py:994: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



### Infer on Schema

In [0]:
schema = tfdv.infer_schema(statistics=train_stats)

In [0]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'so2',FLOAT,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'soot',FLOAT,required,,-


### Calculate the descriptive statistics of the testing subset


In [0]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)

## Step 4: Compare statistics of test subset with schema

### Search for anomalies on new data

In [0]:
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

### Show all detected anomalies

- Int number more than 10
- STRING type when INT expected
- FLOAT type when INT expected
- Int number lower than 0

In [0]:
tfdv.display_anomalies(anomalies)

### New data with anomalies

In [0]:
test_set_copy = test_set.copy()

In [0]:
test_set_copy.drop("soot", axis=1, inplace=True)

### Statistics based on anomalies Data

In [0]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)

In [0]:
anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema)

In [0]:
tfdv.display_anomalies(anomalies_new)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


## Step 5: Prepare schema to upload on Production

In [0]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

### Delete objective column of production schema

In [0]:
tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

### Test anomalies between server environment and new input data

In [0]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment="SERVING")

In [0]:
tfdv.display_anomalies(serving_env_anomalies)

## Step 6: Freeze schema

In [0]:
tfdv.write_schema_text(schema = schema, output_path = "pollution_schema.pbtxt")